In [259]:
import pandas as pd
import numpy as np
from psaw import PushshiftAPI

from tqdm import tqdm
tqdm.pandas()

import datetime

# Article Data Loading 

In [119]:
art_data = pd.read_csv('../../data/webpage_data/full_flipside_data.csv')

In [120]:
art_data.head()

,article_url,date,title,left_sum,right_sum,linked_arts
0,https:/theflipside.io/archives/checks-in-the-mail,"March 20, 2020",Checks In The Mail,The left supports cash payments and argues tha...,The right is generally supportive of helping c...,['https://apnews.com/0373a78b0aacc49d1f6bd8901...
1,https:/theflipside.io/archives/eos-regarding-t...,"January 26, 2021",EOs Regarding Transgender Rights,The left is supportive of both policies.,The right is critical of both policies.,['https://apnews.com/article/biden-reverse-ban...
2,https:/theflipside.io/archives/facebook-and-br...,"April 2, 2019",Facebook and Breaking Up Big Tech,"The left is skeptical of Zuckerberg’s motives,...",The right is disturbed by the free speech impl...,['https://www.washingtonpost.com/opinions/mark...
3,https:/theflipside.io/archives/general-electio...,"October 28, 2020",General Election Update,The left is optimistic about Biden’s chances.,The right is cautiously optimistic about Trump...,['https://theflipside.us15.list-manage.com/tra...
4,https:/theflipside.io/archives/impeachment-hea...,"November 22, 2019",Impeachment Hearings Continue,"The left supports impeachment, arguing that th...","The right opposes impeachment, arguing that th...",['https://theflipside.us15.list-manage.com/tra...


In [208]:
queries = art_data[['title', 'date']]
queries = queries.values

In [206]:
queries.values

array([[' Checks In The Mail', 'March 20, 2020'],
       [' EOs Regarding Transgender Rights', 'January 26, 2021'],
       [' Facebook and Breaking Up Big Tech', 'April 2, 2019'],
       ...,
       ['Arkansas Bans Transgender Treatment for Children',
        'April 8, 2021'],
       ['Afghanistan and the World', 'August 19, 2021'],
       ['Biden Visits Saudi Arabia', 'July 15, 2022']], dtype=object)

# Reddit Scraping 

In [199]:
api = PushshiftAPI()

In [200]:
gen = api.search_submissions(subreddit = 'politics', q = 'checks in the mail', after = int(min_date), before = int(max_date))

In [50]:
MAX_COLS = 74 # Maximum number of columns returned for submissions
while True:
    data = next(gen).d_
    if len(data) >= MAX_COLS:
        break

C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


In [51]:
COLUMNS = list(data.keys())

In [220]:
def get_posts_by_news(query, output_csv, date_thresh = 10, header = True, max_posts = 10):
    title = query[0]
    date = datetime.datetime.strptime(query[1], '%B %d, %Y')
    
    min_date = (date - datetime.timedelta(days = date_thresh)).timestamp()
    max_date = (date + datetime.timedelta(days = date_thresh)).timestamp()
    
    gen = api.search_submissions(subreddit = 'politics', q = title, after = int(min_date), before = int(max_date))
    
    for i, post in enumerate(gen):
        data = post.d_
        data_df = pd.DataFrame({key:str(val) for key, val in data.items()}, index = [i], columns = COLUMNS).fillna('None')
        data_df['title_q'] = title
        data_df['date_q'] = date
        
        data_df.to_csv(output_csv, mode = 'a', escapechar = '\\', header = header)
        
        if i>max_posts:
            br

In [225]:
OUTPUT_CSV = '../../data/reddit_data/reddit_scrape.csv'
WRITE_HEAD = False
for i, query in enumerate(queries):
    get_posts_by_news(query, OUTPUT_CSV, header = WRITE_HEAD)
    print(f'Finished query {i} of {len(queries)}', end = '\r')

C:\Users\ndeas\miniconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)


In [226]:
reddit_posts = pd.read_csv(OUTPUT_CSV)

C:\Users\ndeas\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (2,27,30,67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [235]:
reddit_posts.groupby('title_q').count().sort_values('author', ascending = False).head(50)

,Unnamed: 0,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,title,total_awards_received,treatment_tags,upvote_ratio,url,url_overridden_by_dest,whitelist_status,wls,created,date_q
title_q,,,,,,,,,,,,,,,,,,,,,
Ukraine,7534,7534,7534,7534,7534,7534,7534,7534,7525,7534,...,7534,7534,7534,7534,7522,7522,7534,7534,7534,7534
Coronavirus,7320,7320,7320,7320,7320,7320,7320,7320,7316,7320,...,7320,7320,7320,7320,7320,7320,7320,7320,7320,7320
Impeachment,5904,5904,5904,5904,5904,5904,5904,5904,5894,5904,...,5904,5904,5904,5904,5904,5904,5904,5904,5904,5904
Kavanaugh,2110,2110,2110,2110,2110,2110,2110,2110,2110,2110,...,2110,2110,2110,2110,2110,2110,2110,2110,2110,2110
Hunter Biden,980,980,980,980,980,980,980,980,979,980,...,980,980,980,980,980,980,980,980,980,980
Joe Biden,916,916,916,916,916,916,916,916,915,916,...,916,916,916,916,916,916,916,916,916,916
State of the Union,912,912,912,912,912,912,912,912,911,912,...,912,912,912,912,912,912,912,912,912,912
Portland,837,837,837,837,837,837,837,837,837,837,...,837,837,837,837,837,837,837,837,837,837
Kenosha,676,676,676,676,676,676,676,676,676,676,...,676,676,676,676,676,676,676,676,676,676


In [236]:
reddit_posts.columns

Index(['Unnamed: 0', 'all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext',
       'author_flair_template_id', 'author_flair_text',
       'author_flair_text_color', 'author_flair_type', 'author_fullname',
       'author_is_blocked', 'author_patreon_flair', 'author_premium',
       'awarders', 'can_mod_post', 'contest_mode', 'created_utc', 'domain',
       'full_link', 'gildings', 'id', 'is_created_from_ads_ui',
       'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_css_class',
       'link_flair_richtext', 'link_flair_text', 'link_flair_text_color',
       'link_flair_type', 'locked', 'media', 'media_embed', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'post_hint',
       'preview', 'pwls', 'removed_by_category'

In [240]:
reddit_short = reddit_posts[['title_q', 'date_q', 'title', 'selftext', 'upvote_ratio']]

In [242]:
NUM_SAMPLE = 5
samples = []
for i, group in reddit_short.groupby('upvote_ratio'):
    group = group.sort_values('upvote_ratio', ascending = False)
    new_group = group.iloc[:NUM_SAMPLE]
    samples.append(new_group)

In [244]:
reddit_short = pd.concat(samples, axis = 0)

In [245]:
reddit_short

,title_q,date_q,title,selftext,upvote_ratio
37784,Impeachment,2021-01-12,"To move on, drop the Trump impeachment trial",NaN,0.01
7953,Hunter Biden,2020-12-11,Hunter Biden emailed best wishes from the ‘ent...,[deleted],0.02
7960,Hunter Biden,2020-12-11,"Hunter Biden Email Reportedly Shows $400,000 i...",NaN,0.03
17695,Michael Flynn,2020-06-25,Sidney Powell: Michael Flynn Planned to ‘Audit...,NaN,0.03
38863,Impeachment,2021-01-12,Trump faces impeachment again as Democrats mak...,NaN,0.03
...,...,...,...,...,...
0,Checks In The Mail,2020-03-20,Coronavirus relief checks could take four mont...,NaN,None
39561,Impeachment,2020-01-16,Nadler Says Trump Acting Like ‘a Dictator’: Im...,NaN,None
39570,Impeachment,2020-01-16,The impeachment evidence will catch up to Repu...,NaN,None
39569,Impeachment,2020-01-16,Democrats feel political momentum swinging to ...,NaN,None


In [246]:
titles = reddit_short['title_q'].unique()

In [247]:
titles

array(['Impeachment', 'Hunter Biden', 'Michael Flynn',
       'Biden-Putin Summit', 'Merrick Garland', 'Statues', 'Debt Ceiling',
       'Obamagate', 'Dr. Seuss', 'Minimum Wage',
       'Climate Executive Orders', 'Neera Tanden', 'Portland',
       'Biden’s Speech', 'Marjorie Taylor Greene', 'Airstrikes in Syria',
       'Critical Race Theory', 'HEROES Act',
       'Supreme Court on Religious Schools', 'Amy Coney Barrett',
       'Biden’s Inauguration', 'Equality Act', 'Infrastructure Plan',
       'Immigration', 'Police Protests', 'Jeff Sessions', 'GameStop',
       'Police Reform', 'Jobs Report', 'Ukraine', 'Ketanji Brown Jackson',
       'Russia-Taliban Bounties', 'Education Commission',
       'Trump Signs Executive Order', 'COVID Vaccine',
       'Afghanistan and Biden', 'COVID Stimulus',
       'Immigration Restrictions', 'Pete Buttigieg', 'Elon Musk',
       'Bob Dole', 'Bahrain-Israel Deal', 'Wildfires', 'Andrew Cuomo',
       'Juneteenth', 'COVID Relief Bill', 'G7 Summit', 'Ah

In [248]:
src_titles = queries[:, 0]

In [252]:
miss_titles = set(src_titles).difference(titles)

In [284]:
miss_queries = [list(query) for query in queries if query[0] in miss_titles]

In [ ]:
OUTPUT_CSV = '../../data/reddit_data/reddit_scrape.csv'
WRITE_HEAD = False
for i, query in enumerate(miss_queries):
    get_posts_by_news(query, OUTPUT_CSV, date_thresh = 30, header = WRITE_HEAD, max_posts = 10)
    print(f'Finished query {i} of {len(queries)}', end = '\r')